Joel Gritter & Kindeep Singh Kargil

COSC 4P80 - Seminar Demo

March 29, 2021

In [3]:
# imports for data manipulation
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# imports for machine learning
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [4]:
# read in the notes, make an array with 0's, except for the current note
def read_and_format(input_filepath):
  input_data = []
  with open(input_filepath) as input_file:
    for line in input_file:
      values = line.split(",")
      for value in values:
        tmp = [0.0] * 88
        v = int(value)
        tmp[v-1] = 1.0
        input_data.append(tmp)
  
  return input_data
  

input_data = read_and_format("k330-allegro-moderato.csv")

In [5]:
# get the previous 20 notes, predict the next note
def generate_datasets(input_array, n_prev = 20):
  temp_x = [input_array[i:i+n_prev] for i in range(len(input_array) - n_prev)]
  temp_y = [input_array[i+n_prev] for i in range(len(input_array) - n_prev)]

  return np.array(temp_x), np.array(temp_y)

x, y = generate_datasets(input_data)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, shuffle=True)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(y_train[0])

(2008, 20, 88) (2008, 88)
(502, 20, 88) (502, 88)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [6]:
# build the model itself
model = Sequential()
model.add(LSTM(30))
model.add(Dense(88, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# train the model
model.fit(x_train, y_train, batch_size=10, epochs=100, validation_split=0.05)

Epoch 1/100
191/191 [==============================] - 4s 11ms/step - loss: 3.7037 - accuracy: 0.1409 - val_loss: 2.8201 - val_accuracy: 0.1485
Epoch 2/100
191/191 [==============================] - 1s 8ms/step - loss: 2.6582 - accuracy: 0.1591 - val_loss: 2.8059 - val_accuracy: 0.1485
Epoch 3/100
191/191 [==============================] - 1s 8ms/step - loss: 2.6458 - accuracy: 0.1698 - val_loss: 2.7629 - val_accuracy: 0.1485
Epoch 4/100
191/191 [==============================] - 1s 7ms/step - loss: 2.6282 - accuracy: 0.1714 - val_loss: 2.7299 - val_accuracy: 0.1485
Epoch 5/100
191/191 [==============================] - 1s 8ms/step - loss: 2.5811 - accuracy: 0.1817 - val_loss: 2.6803 - val_accuracy: 0.1881
Epoch 6/100
191/191 [==============================] - 2s 8ms/step - loss: 2.5178 - accuracy: 0.2112 - val_loss: 2.5997 - val_accuracy: 0.2079
Epoch 7/100
191/191 [==============================] - 2s 8ms/step - loss: 2.4505 - accuracy: 0.2361 - val_loss: 2.5154 - val_accuracy: 0.257

In [8]:
# test the model
model.evaluate(x_test, y_test)

16/16 [==============================] - 0s 3ms/step - loss: 1.3169 - accuracy: 0.6614


[1.3169466257095337, 0.6613546013832092]

In [9]:
# See incorrectly predicted 
predictions = model.predict(x_test)
incorrect_indices = []
for (index, (prediction, target)) in enumerate(zip(predictions, y_test)):
  pred = np.argmax(prediction)
  tar = np.argmax(target)
  if pred != tar:
    incorrect_indices.append(index)

print(", ".join(map(str, incorrect_indices)))

7, 8, 10, 11, 12, 16, 19, 22, 23, 25, 26, 27, 30, 31, 32, 34, 37, 44, 45, 46, 50, 53, 54, 56, 62, 66, 72, 73, 74, 79, 84, 91, 95, 97, 98, 99, 111, 115, 116, 119, 121, 122, 126, 129, 130, 131, 134, 136, 140, 141, 146, 148, 149, 156, 167, 170, 172, 174, 177, 187, 191, 196, 198, 203, 206, 210, 215, 217, 220, 221, 222, 227, 228, 233, 234, 239, 248, 251, 252, 256, 257, 258, 259, 262, 265, 268, 269, 272, 273, 276, 279, 281, 282, 287, 289, 292, 296, 298, 305, 308, 309, 311, 313, 317, 322, 325, 327, 330, 337, 338, 341, 342, 346, 349, 350, 356, 358, 359, 360, 362, 366, 367, 370, 372, 373, 375, 376, 377, 380, 381, 383, 386, 390, 393, 394, 395, 396, 398, 400, 402, 408, 409, 417, 419, 420, 425, 428, 429, 430, 435, 436, 442, 445, 447, 451, 453, 455, 459, 463, 466, 470, 472, 483, 488, 490, 493, 496, 498, 499, 500


In [10]:
# Predict song
test_in = x_test[0]

# initial - provide inital 20 notes
# n - how many predicted notes to add (i.e. expand by this number)
def make_big_song(initial, n):
  res =[ x for x in initial]
  for _ in range(n):
    next = model.predict(np.array([res[-20:],]))[0]
    res.append(next)

  return np.array(res)

test = make_big_song(test_in, 20)
print(test.shape)

(40, 88)
